In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [2]:
import tensorflow as tf
from tensorflow import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.10.0
Keras version: 2.10.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
data_gen = ImageDataGenerator( rescale=1./255)
trdata = data_gen.flow_from_directory('../data/processed/train/', target_size=(200, 200), class_mode='sparse', shuffle=True)
tsdata = data_gen.flow_from_directory('../data/processed/test/', target_size=(200, 200), class_mode='sparse')

Found 22224 images belonging to 2 classes.
Found 2776 images belonging to 2 classes.


In [4]:
model = Sequential()

model.add(Conv2D(64, (3,3), padding="same", activation="relu", input_shape=(200,200,3)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))

model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))

model.add(Conv2D(256, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))

model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))

model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(Conv2D(512, (3,3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2)))

model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))              
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))              
model.add(Dense(2, activation="softmax"))

In [5]:
model.compile(optimizer="adam", loss= SparseCategoricalCrossentropy(from_logits = False), metrics=["accuracy"])
print("Class index:", trdata.class_indices)
print("Samples train:", trdata.samples, "batch_size:", trdata.batch_size)

Class index: {'cats': 0, 'dogs': 1}
Samples train: 22224 batch_size: 32


In [6]:
stop = EarlyStopping(monitor= 'val_loss', patience= 5, restore_best_weights= True)
reduce = ReduceLROnPlateau(monitor= 'val_loss', factor= 0.3, patience= 3)
chp = ModelCheckpoint("../models/best_model.h5", monitor="val_accuracy", save_best_only=True, mode="max", verbose=1)

In [8]:
hst = model.fit(trdata, validation_data= tsdata, epochs= 40, callbacks=[stop, reduce, chp])

Epoch 1/40
695/695 [==============================] - 293s 416ms/step - loss: 1.5911 - accuracy: 0.5419 - val_loss: 0.8226 - val_accuracy: 0.5317 - lr: 0.0010
Epoch 2/40
695/695 [==============================] - 299s 430ms/step - loss: 0.7108 - accuracy: 0.5687 - val_loss: 0.6782 - val_accuracy: 0.5987 - lr: 0.0010
Epoch 3/40
695/695 [==============================] - 300s 431ms/step - loss: 0.7135 - accuracy: 0.5638 - val_loss: 0.6627 - val_accuracy: 0.6308 - lr: 0.0010
Epoch 4/40
695/695 [==============================] - 300s 431ms/step - loss: 0.7194 - accuracy: 0.5579 - val_loss: 0.7097 - val_accuracy: 0.5213 - lr: 0.0010
Epoch 5/40
695/695 [==============================] - 301s 433ms/step - loss: 0.7210 - accuracy: 0.5451 - val_loss: 0.6911 - val_accuracy: 0.5375 - lr: 0.0010
Epoch 6/40
695/695 [==============================] - 299s 431ms/step - loss: 0.7069 - accuracy: 0.5173 - val_loss: 0.6937 - val_accuracy: 0.4939 - lr: 0.0010
Epoch 7/40
695/695 [==========================

In [10]:
#save the model

model.save('../models/model.h5')